<a href="https://colab.research.google.com/github/osmartinez/modelizacion-placa-calor/blob/master/placa_calor_jacobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math

In [0]:
'''
Función que descompone A
D + R = A
'''
def descomponer_matriz(A):
  p,q = A.shape
  D = np.zeros(shape=(p,q))
  R= np.zeros(shape=(p,q))
  
  for i in range(p):
    for j in range(q):
      if i == j:
        D[i][j] = A[i][j]
        R[i][j] = 0
      else:
        D[i][j] = 0
        R[i][j] = A[i][j]
  
  return D,R

In [0]:
'''
Matriz que invierte una matriz diagonal D
'''
def invertir_diagonal(D):
  p,q = D.shape
  D_inv = np.zeros(shape=(p,q))
  
  for i in range(p):
    for j in range (q):
      if i == j:
        D_inv[i][j] = 1/D[i][j]
        
  return D_inv

In [0]:
'''
Aplica método de Jacobi al sistema de ecuaciones Ax=b,
descomponiendo A en R+D
Utiliza como criterio de parada un número N de iteraciones
'''
def do_jacobi_secuencial_descomp_n(A, b, N=100):
  dim_b = b.shape
  dim_A = A.shape
  p,q = dim_A
  
  if(dim_b[0] != dim_A[0]):
    raise Exception('Dimensiones no compatibles')
  
  D,R = descomponer_matriz(A)
  D_inv = invertir_diagonal(D)
  
  sol = np.zeros(shape=dim_b)
  
  # itero N veces el método
  for n in range(N):
    sol = np.dot(D_inv, (b- np.dot(R,sol)) )
  
  return sol

In [0]:
'''
Aplica método de Jacobi secuencialmente al sistema Ax= b,
sin descomposición
Utiliza como criterio de parada N iteraciones
'''
def do_jacobi_secuencial_n(A, b, N=100):
  p,q = A.shape
  dim_b = b.shape
  
  if(dim_b[0] != p):
    raise Exception('Dimensiones no compatibles')
    
  sol = np.zeros(shape = dim_b)
  
  for n in range(N):
    for i in range (p):
      elemento_diag= A[i][i]
      numerador = b[i]
      for j in range(q):
        if j == i:
          continue
        numerador-=A[i][j]*sol[j]
      sol[i] = numerador/elemento_diag
  return sol

In [0]:
'''
Aplica método de Jacobi secuencialmente al sistema Ax= b,
sin descomposición
Utiliza como criterio de parada N iteraciones o tolerancia al error
'''
def do_jacobi_secuencial_n_tol(A, b, N=100, tol=1e-10):
  p,q = A.shape
  dim_b = b.shape
  
  if(dim_b[0] != p):
    raise Exception('Dimensiones no compatibles')
    
  sol = np.zeros(shape = dim_b)
  sol_actual = np.zeros(shape = dim_b)
  
  for n in range(N):
    for i in range (p):
      elemento_diag= A[i][i]
      numerador = b[i]
      for j in range(q):
        if j == i:
          continue
        numerador-=A[i][j]*sol[j]
      sol_actual[i] = numerador/elemento_diag
      
    suma =0
    for i in range (p):
      suma += math.sqrt(math.pow(sol_actual[i]-sol[i],2))
      sol[i] = sol_actual[i]
    
    if suma<tol:
      print('Tolerancia alcanzada! Iteración:'+str(n))
      break      
      
  return sol

In [134]:
A = np.array([[5,2,1,1],
              [2,6,2,1],
              [1,2,7,1],
              [1,1,2,8]])

b = np.array([29,31,26,19])

print (do_jacobi_secuencial_n_tol(A,b,100,1e-11))
print (do_jacobi_secuencial_descomp_n(A,b,100))
print (do_jacobi_secuencial_n(A,b,100))

Tolerancia alcanzada! Iteración:75
[3.99275362 2.95410628 2.16183575 0.96618357]
[3.99275362 2.95410628 2.16183575 0.96618357]
[3.99275362 2.95410628 2.16183575 0.96618357]
